In [1]:
import pandas as pd
import gzip
import pickle
import os
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVC


In [2]:

# Paso 1.
# Realice la limpieza de los datasets:
def limpiar_dataset(df):
    # - Renombre la columna "default payment next month" a "default".
    df = df.rename(columns={
        'default payment next month': 'default'
    })

    # - Remueva la columna "ID".
    df = df.drop('ID', axis= 1)

    # - Elimine los registros con informacion no disponible.
    df = df[df['EDUCATION']>0]
    df = df[df['MARRIAGE']>0]

    # - Para la columna EDUCATION, valores > 4 indican niveles superiores
    #   de educación, agrupe estos valores en la categoría "others".
    df.loc[df['EDUCATION'] > 4 , 'EDUCATION'] = 4    
    return df

In [3]:
train = pd.read_csv('../files/input/train_data.csv.zip')
test = pd.read_csv('../files/input/test_data.csv.zip')

In [4]:
train.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,10748,310000,1,3,1,32,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,12574,10000,2,3,1,49,-1,-1,-2,-1,...,1690,1138,930,0,0,2828,0,182,0,1
2,29677,50000,1,2,1,28,-1,-1,-1,0,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,8857,80000,2,3,1,52,2,2,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,21099,270000,1,1,2,34,1,2,0,0,...,22448,15490,17343,0,4000,2000,0,2000,2000,0


In [5]:
train = limpiar_dataset(train)
test = limpiar_dataset(test)
#Paso 2: 
# Divida los datasets en x_train, y_train, x_test, y_test.
X_train = train.drop('default', axis=1)  # variables independientes
y_train = train['default'] # variable objetivo
X_test = test.drop('default', axis=1)  # variables independientes
y_test = test['default'] # variable objetivo



In [6]:
# (21000, 23)
X_train.shape

(20953, 23)

In [7]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
# - Estandariza la matriz de entrada.
# - Selecciona las K columnas mas relevantes de la matrix de entrada.
# - Ajusta una maquina de vectores de soporte (svm).

# Variables categóricas
cat_features = ['SEX', 'EDUCATION', 'MARRIAGE']
numeric_features = [
    'LIMIT_BAL', 'AGE',
    'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
    'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'
]
# Transformador: aplica OneHotEncoder a las categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        ('num', StandardScaler(), numeric_features) # escalado de 0-1
    ],
    remainder='passthrough'  # deja las numéricas igual
)

# Crear el pipeline con preprocesamiento + modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', PCA()),                              # usa todas las componentes
    # ('scaler', StandardScaler()),                # estandariza la matriz (mejor se hace una sola vez arriba y no en el pipeline)
    ('select', SelectKBest(score_func=f_classif)), 
    ('model', SVC())
])

# Entrenar el pipeline
pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('pca', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [15]:
## Paso 4.
# Optimice los hiperparámetros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada.
# Use la función de precisión balanceada para medir la precisión del modelo.

# Definir la grilla de hiperparámetros
param_grid = {
    'select__k': [17],
    'pca__n_components': [None],
    # 'model__gamma': [1],
    'model__gamma': ['scale', 'auto', 0.01, 0.1, 1],

}

# Definir la métrica y el método de validación cruzada
scorer = make_scorer(balanced_accuracy_score)

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring=scorer,
    cv=10,              # 10 splits de validacion cruzada
    n_jobs=-1,
    verbose=2
)

# Ejecutar la búsqueda
grid_search.fit(X_train, y_train)

# Mostrar los mejores resultados
print("Mejor precisión balanceada (CV):", grid_search.best_score_)
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)


Fitting 10 folds for each of 5 candidates, totalling 50 fits
Mejor precisión balanceada (CV): 0.6501426702989455
Mejores hiperparámetros encontrados:
{'model__gamma': 0.1, 'pca__n_components': None, 'select__k': 17}


In [16]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

best_model = grid_search

# Crear la ruta donde se guardará el modelo
os.makedirs("../files/models", exist_ok=True)

# Guardar el modelo comprimido
with gzip.open("../files/models/model.pkl.gz", "wb") as f:
    pickle.dump(best_model, f)

print("Modelo guardado en 'files/models/model.pkl.gz'")


Modelo guardado en 'files/models/model.pkl.gz'


In [17]:
model = best_model

In [18]:

# # Ruta del modelo guardado
# model_path = "../files/models/model.pkl.gz"

# # Cargar el modelo
# with gzip.open(model_path, "rb") as f:
#     model = pickle.load(f)

# # print(type(model))

In [19]:


# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calcular métricas
def calcular_metricas(y_true, y_pred, dataset_name):
    return {
        "type": "metrics",
        "dataset": dataset_name,
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1_score": f1_score(y_true, y_pred, zero_division=0)
    }

train_metrics = calcular_metricas(y_train, y_train_pred, "train")
test_metrics = calcular_metricas(y_test, y_test_pred, "test")

# Guardar las métricas en JSON
os.makedirs("../files/output", exist_ok=True)
output_path = "../files/output/metrics.json"

with open(output_path, "w", encoding="utf-8") as f:
    for entry in [train_metrics, test_metrics]:
        f.write(json.dumps(entry) + "\n")

print("Métricas guardadas en:", output_path)
# print(json.dumps([train_metrics, test_metrics], indent=4))
#
# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#
def matriz_confusion_dict(y_true, y_pred, dataset_name):
    cm = confusion_matrix(y_true, y_pred)
    # Estructura de la matriz de confusión:
    # [[TN, FP],
    #  [FN, TP]]
    return {
        "type": "cm_matrix",
        "dataset": dataset_name,
        "true_0": {
            "predicted_0": int(cm[0][0]),
            "predicted_1": int(cm[0][1])
        },
        "true_1": {
            "predicted_0": int(cm[1][0]),
            "predicted_1": int(cm[1][1])
        }
    }

train_cm = matriz_confusion_dict(y_train, y_train_pred, "train")
test_cm = matriz_confusion_dict(y_test, y_test_pred, "test")
metrics_path = "../files/output/metrics.json"

# Si ya existen métricas guardadas, las cargamos
metrics_data = []
if os.path.exists(metrics_path):
    with open(metrics_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # ignora líneas vacías
                metrics_data.append(json.loads(line))
            # metrics_data = json.load(f)
else:
    metrics_data = []

# Agregar las matrices de confusión
metrics_data.extend([train_cm, test_cm])

# Guardar todo nuevamente en el archivo JSON
with open(metrics_path, "w", encoding="utf-8") as f:
    for entry in metrics_data:
        f.write(json.dumps(entry) + "\n")
print("Matrices de confusión agregadas al archivo:", metrics_path)

# print("Métricas y matrices guardadas en formato JSONL en:", output_path)


Métricas guardadas en: ../files/output/metrics.json
Matrices de confusión agregadas al archivo: ../files/output/metrics.json


In [20]:
model.score(X_train, y_train)
#     0.661,


0.6650091291686067

In [21]:
model.score(X_test, y_test)
#     0.666,

0.6680497595974464